In [ ]:
#!/usr/bin/env python3
"""
Main file to test the helper functions.
"""
import contextily as cx
import math
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd

from helpers.hurricane_helpers import (
    get_hurricane_bin_midpoint_times,
    get_hurricane_bin_start_times,
    get_hurricane_bin_end_times,
    list_all_hurricanes,
    interpolate_besttrack_for_code,
    interpolate_all_hurricanes_besttrack
)
from helpers.ships_helpers import save_ships_data, interpolate_ships_info_for_hurricane, interpolate_all_hurricanes_ships
from helpers.glm_helpers import process_all_hurricanes_glm, process_glm_info_for_hurricane

from helpers.orchestration_helpers import download_all_data_for_hurricane, download_all_data_for_all_hurricanes

In [ ]:
print(list_all_hurricanes())

In [ ]:
download_all_data_for_hurricane("AL092022")

In [ ]:
def plot_bin_lightning(final_df, bin_start, bin_end, center_lat, center_lon, box_size):
    '''
    Plot lightning groups in vicinity of hurricane center

    Args:
        final_df: Dataframe containing lightning group data within time bin
        bin_start: Start datetime of bin
        bin_end: End datetime of bin
        center_lat: Latitude of hurricane center to draw on plot
        center_lon: Longitude of hurricane center to draw on plot
        box_size: Size of box around hurricane center to show in plot

    Returns:
        None (just shows plot)
    '''
    fig, ax = plt.subplots(figsize=(8,8))
    plt.scatter(final_df['Group Longitude'], final_df['Group Latitude'], s=2, label='Lightning Group')
    plt.scatter(center_lon, center_lat, color='black', label='Hurricane Center')
    plt.xlim(center_lon - box_size, center_lon + box_size)
    plt.ylim(center_lat - box_size, center_lat + box_size)
    plt.xlabel('Longitude (deg)')
    plt.ylabel('Latitude (deg)')
    cx.add_basemap(ax, crs="EPSG:4326", source=cx.providers.OpenStreetMap.Mapnik)
    plt.title(f'Lightning Groups Near Hurricane Ian Center, {bin_start}-{bin_end}')
    plt.legend()
    plt.show()

In [ ]:
def plot_shear_dist(dens_by_bin, bin_start, bin_end, bin_size_radial_rmw_mult=0.25, n_bins_radial = 6, n_bins_angular = 12):
    #create density by bin array
    # define binning
    rbins = np.linspace(0, n_bins_radial * bin_size_radial_rmw_mult, n_bins_radial + 1)
    abins = np.linspace(0,2*np.pi, n_bins_angular + 1)

    #print(rbins)
    #print(abins)

    #split each angular bin in 10 to avoid problems with straight lines
    abins_fine = np.linspace(
        0,
        2*np.pi,
        n_bins_angular * 100 + 1
    )

    #upsample density array for larger list of angular bins
    dens_fine = np.repeat(dens_by_bin, 100, axis=1)

    # plot
    fig, ax = plt.subplots(subplot_kw=dict(projection="polar"))

    #show angles using geographic convention, not mathematical
    ax.set_theta_zero_location("N")
    ax.set_theta_direction(-1)  

    #print(dens_by_bin.T.shape)
    pc = ax.pcolormesh(abins_fine, rbins, dens_fine, cmap="magma_r")
    fig.colorbar(pc)

    fig.suptitle(f'Lightning by Distance and Direction from Storm Center, Hurricane Ian {bin_start}-{bin_end}')
    plt.show()


In [ ]:
#Hurricane code for current storm (Ian)
ian_hurricane_code = "AL092022"

#Read GLM data
ian_glm = pd.read_csv('data/storms/IAN_2022/glm/groups.csv')

#Read best track data
ian_besttrack = pd.read_csv('data/storms/IAN_2022/hurricane/besttrack.csv')

#Read SHIPS data
ian_ships = pd.read_csv('data/storms/IAN_2022/ships/ships_interpolated.csv')

# Get bin times
bin_times = get_hurricane_bin_midpoint_times(ian_hurricane_code, region='atl', time_interval=30)
bin_starts = get_hurricane_bin_start_times(ian_hurricane_code, region='atl', time_interval=30)
bin_ends = get_hurricane_bin_end_times(ian_hurricane_code, region='atl', time_interval=30)

#Loop through bins
for idx in range(len(bin_times)):
    #Bin times
    curr_bin_time = str(bin_times[idx])
    print(curr_bin_time)

    curr_bin_start = str(bin_starts[idx])
    curr_bin_end = str(bin_ends[idx])

    #Filtered GLM and best track data (join)
    ian_glm_bin = ian_glm[ian_glm["Bin Time"] == curr_bin_time]
    #If no lightning is found, don't go further
    if ian_glm_bin.empty:
        print("No lightning found.")
        continue

    #print(ian_glm_bin)
    ian_besttrack_bin = ian_besttrack[ian_besttrack["Timestamp"] == curr_bin_time]
    #print(ian_besttrack_bin)
    ian_bin_join = pd.merge(ian_glm_bin, ian_besttrack_bin, left_on = "Bin Time", right_on = "Timestamp")

    #Filtered SHIPS data (join)
    ian_ships_bin = ian_ships[ian_ships["Timestamp"] == curr_bin_time]
    ian_bin_join = pd.merge(ian_bin_join, ian_ships_bin, left_on = "Bin Time", right_on = "Timestamp")

    #print(ian_bin_join.head)
    #Only move forward if we have wind shear data at this timestamp
    if np.isnan(ian_bin_join["Shear Direction (deg)"].iloc[0]):
        continue
    
    print (f'Shear Direction: {ian_bin_join["Shear Direction (deg)"].iloc[0]}')
    
    #Extract RMW
    #print(ian_bin_join)
    curr_rmw = ian_besttrack_bin["Radius of Maximum Winds"].iloc[0]
    #print(curr_rmw)

    #Extract center latitude and longitude
    curr_center_lat = ian_besttrack_bin["Latitude"].iloc[0]
    curr_center_lon = ian_besttrack_bin["Longitude"].iloc[0]

    #Add fields to mark shear direction and distance as a multiple of RMW
    ian_bin_join["Direction From Hurricane Center Relative To Wind Shear (deg)"] = ((ian_bin_join["Direction from Hurricane Center (deg)"] - ian_bin_join["Shear Direction (deg)"]) % 360)

    ian_bin_join["Distance From Hurricane Center (kt)"] = ian_bin_join["Distance From Hurricane Center (m)"] / 1852
    ian_bin_join["Distance_RMW_Mult"] = ian_bin_join["Distance From Hurricane Center (kt)"] / curr_rmw
    #print(ian_bin_join)

    #Plot binning parameters
    n_bins_radial = 1
    n_bins_angular = 12
    bin_size_radial_rmw_mult = 1.5
    bin_size_angular = 360 / n_bins_angular

    group_counts = np.zeros((n_bins_radial, n_bins_angular))
    dens_by_bin = np.zeros((n_bins_radial, n_bins_angular))

    #Find density of each bin
    for rad in range(n_bins_radial):
        for ang in range(n_bins_angular):
            #print(rad, ang)
            min_dist = rad * bin_size_radial_rmw_mult
            max_dist = (rad + 1) * bin_size_radial_rmw_mult
            min_ang = ang * bin_size_angular
            max_ang = (ang + 1) * bin_size_angular
            #print(min_dist, max_dist, min_ang, max_ang)

            glm_mask = (
                        (ian_bin_join["Distance_RMW_Mult"] >= min_dist) & (ian_bin_join["Distance_RMW_Mult"] < max_dist) &
                        (ian_bin_join["Direction From Hurricane Center Relative To Wind Shear (deg)"] >= min_ang) & (ian_bin_join["Direction From Hurricane Center Relative To Wind Shear (deg)"] < max_ang)
                       )
            #glm_mask = (
            #            (ian_bin_join["Distance_RMW_Mult"] >= min_dist) & (ian_bin_join["Distance_RMW_Mult"] < max_dist) &
            #            (ian_bin_join["Direction from Hurricane Center (deg)"] >= min_ang) & (ian_bin_join["Direction from Hurricane Center (deg)"] < max_ang)
            #        )
        
            curr_glm_mask = ian_bin_join[glm_mask]
            #print(curr_glm_mask)

            curr_ngroups = len(curr_glm_mask)
            curr_bin_area = (1/n_bins_angular) * math.pi*((max_dist*curr_rmw*1.852)**2 - (min_dist*curr_rmw*1.852)**2)

            curr_density = curr_ngroups/curr_bin_area
            #print(curr_ngroups, curr_bin_area, curr_density)

            dens_by_bin[rad, ang] = curr_density
            group_counts[rad, ang] = curr_ngroups

    plot_shear_dist(dens_by_bin, curr_bin_start, curr_bin_end, bin_size_radial_rmw_mult, n_bins_radial, n_bins_angular)
    plot_bin_lightning(ian_bin_join[ian_bin_join["Distance_RMW_Mult"] < 1.5], curr_bin_start, curr_bin_end, curr_center_lat, curr_center_lon, curr_rmw/60*2)

In [ ]:
print(ian_glm["Bin Time"].dtype)
print(ian_besttrack["Timestamp"].dtype)
print(type(curr_bin_time))